# Datenbeschaffung Adliswil

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from locale import setlocale, LC_TIME
import os
import re
from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# Wir starten den Browser
driver = webdriver.Firefox()

# Wir sagen dem Browser, welche Seite er besuchen sollte.
driver.get('https://www.adliswil.ch/politbusiness?politische_geschaefte_suchformular%5Bkeyword%5D=&politische_geschaefte_suchformular%5Bnummer%5D=&politische_geschaefte_suchformular%5BvomStart%5D=1.1.2005&politische_geschaefte_suchformular%5BvomEnd%5D=2.1.2024&politische_geschaefte_suchformular%5BstatusId%5D=&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=anfrage&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=dringliches_postulat&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=interpellation&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=motion&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=parlamentarische_initiative&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=postulat&politische_geschaefte_suchformular%5BkategorieId%5D%5B%5D=volksinitiative&politische_geschaefte_suchformular%5BsearchBtn%5D=&politische_geschaefte_suchformular%5B_token%5D=hfH8kTJyMlRgFeP3MuBTf034hQg5kg1Xv41c2EaBKpE')
#Die Hauptseite lädt relativ schnell, 2 Sekunden sollten reichen.
time.sleep(2)
#Es wird später das Problem auftauchen, dass die Elemente auf der Seite beim Laden in unterschiedlicher 
#Reihenfolge angezeigt werden. So werden Elemente mehrmals und andere gar nicht gespeichert. Ich sortiere also
#nach Datum, um das Problem zu beheben.
driver.find_element(By.CSS_SELECTOR, 'th[data-data="_geschaeftsdatum"]').click()

# Nun gebe ich den Befehl, auf jede der 10 Seiten zuzugreifen.
for page_number in range(10):
    #Als nächstes muss ich auf die 20 Subpages zugreifen, die jeweils auf einer der 26 Seiten sind.
    for link_number in range(20):
        #Den Namen der Elemente suche ich über das Entwicklertool
        elements = driver.find_elements(By.XPATH,'//td[@class="icms-datatable-col-title"]/a')
        
    #Es gibt auf der Seite noch ein Weiteres Element mit dem gesuchten Namen. Das soll er ignorieren, 
    #deshalb die Try-Funktion.
        try:
            elements[link_number].click()
            time.sleep(6)
            #Ich speichere die Subpages in einem Ordner als htm Datei ab
            subpage = driver.page_source.encode('utf-8')
            with open("subpages_Adliswil/"+str(page_number)+'_'+str(link_number)+".htm", "wb+") as file:
                file.write(subpage)

            #Gehe zurück zur vorherigen Seite
            driver.back()
        #Hier überspringe ich das falsche Element.
        except Exception as e:
            print(f"Error: {e}")
            continue
        #Es dauert auch hier etwas
        time.sleep(3)

    #Dann soll das Programm zu nächsten Seite gehen. und in der Schlaufe oben wieder beginnen. Die Webseite
    #weist die aktuelle Seitenzahl leider nicht in der URL aus. Ich muss die Seite also mit einem Selenium-Befehl
    #wechseln...
    driver.find_element(By.ID, 'icmsTable-geschaefteList_next').click()
    time.sleep(4)

#Am Ende soll das Programm den Browser schliessen
driver.quit()

Error: Message: Browsing context has been discarded



NoSuchWindowException: Message: Browsing context has been discarded


Die htm-Files sind gespeichert. Nun werden sie in ein CSV zusammengefasst.

In [4]:
#Danach sage ich, wo die Dokumente zu finden sind und schaue, wie viele es sind.
lst = os.listdir("subpages_adliswil/")
len(lst)

199

In [5]:
#Ich schaue mir die Elemente mal an.
for elem in tqdm(lst):
    print(elem)

100%|█████████████████████████████████████| 199/199 [00:00<00:00, 133482.57it/s]

3_13.htm
7_9.htm
1_16.htm
9_6.htm
9_7.htm
1_17.htm
7_8.htm
3_12.htm
3_10.htm
4_19.htm
5_8.htm
9_5.htm
1_15.htm
8_19.htm
8_18.htm
1_14.htm
9_4.htm
5_9.htm
3_11.htm
4_18.htm
1_9.htm
3_15.htm
.DS_Store
9_0.htm
1_10.htm
6_19.htm
1_11.htm
6_18.htm
9_1.htm
3_14.htm
1_8.htm
3_16.htm
3_8.htm
1_13.htm
9_3.htm
9_2.htm
1_12.htm
3_9.htm
3_17.htm
8_5.htm
4_8.htm
7_14.htm
9_11.htm
5_11.htm
2_18.htm
5_10.htm
2_19.htm
9_10.htm
7_15.htm
4_9.htm
8_4.htm
6_9.htm
8_6.htm
7_17.htm
9_12.htm
5_12.htm
5_13.htm
9_13.htm
7_16.htm
8_7.htm
6_8.htm
8_3.htm
9_17.htm
7_12.htm
2_8.htm
5_17.htm
5_16.htm
2_9.htm
7_13.htm
9_16.htm
8_2.htm
8_0.htm
9_14.htm
7_11.htm
0_18.htm
5_14.htm
0_9.htm
0_8.htm
5_15.htm
7_10.htm
0_19.htm
9_15.htm
8_1.htm
6_3.htm
4_1.htm
0_14.htm
2_7.htm
2_11.htm
5_18.htm
0_5.htm
0_4.htm
2_10.htm
5_19.htm
2_6.htm
0_15.htm
4_0.htm
6_2.htm
6_0.htm
4_2.htm
0_17.htm
2_4.htm
2_12.htm
0_6.htm
0_7.htm
2_13.htm
2_5.htm
0_16.htm
4_3.htm
6_1.htm
6_5.htm
4_7.htm
0_12.htm
2_1.htm
2_17.htm
0_3.htm
0_2.htm
2_16.htm

In [6]:
#Bei den Daten von Wädenswil gab es ein Verstecktes Element namens .DS_Store. Das führt zu Problemen, weil es
#das Format nicht lesen kann. Ich mache auch hier zur Sicherheit einen Befehl, dass er eine solche Datei ignoriert.
#Als erstes erstelle ich einen Platzhalter für meinen Datensatz
datensatz_adliswil = []
for elem in tqdm(lst):
    if not elem.startswith('.DS_Store'):
        ##Nun öffne ich eine Daten nach der anderen und lese sie ein.
        with open(os.path.join('subpages_adliswil', elem), 'r') as file:
            text = file.read()
        #Ich lasse das Ganze mit BeautifulSoup lesen.
        soup = BeautifulSoup(text, 'html.parser')
        #Und definiere alle Elemente, die ich haben will.
        try:
            titel = soup.find(class_='main-title').text.strip()
        except:
            titel=None
        try:    
            geschaeftsart = soup.find(class_='icms-desclist-container').find('dt', string='Geschäftsart').find_next_sibling('dd').text.strip()
        except:
            geschaeftsart=None
        try:    
            status = soup.find(class_='icms-desclist-container').find('dt', string='Status').find_next_sibling('dd').text.strip()
        except:
            status=None
        try:
            datum = soup.find(class_='icms-desclist-container').find('dt', string='Datum').find_next_sibling('dd').text.strip()
        except:
            datum=None
        try:
            #namen = soup.find(class_='icms-desclist-container').find('dt', string='Verfasser/Beteiligte').find_next_sibling('dd').text.strip()
            namen = soup.find(class_='icms-desclist-container').find('dt', string='Verfasser/Beteiligte').find_next_sibling('dd').findChildren()
            namen = [name.get_text(separator="\n") for name in namen]
        except:
            namen=None

        # Danach speichere ich diese Elemente in einem Dictionairy....
        mini_dict = {
            'Titel': titel,
            'Geschaeftsart': geschaeftsart,
            'Status': status,
            'Datum': datum,
            'Namen': namen
        }
        # ...und füge das Mini-Dictionary zur Liste hinzu.
        datensatz_adliswil.append(mini_dict)

# Am Schluss mache ich daraus ein Dataframe mit Pandas.
df = pd.DataFrame(datensatz_adliswil)

100%|█████████████████████████████████████████| 199/199 [00:04<00:00, 48.23it/s]


In [7]:
df

,Titel,Geschaeftsart,Status,Datum,Namen
0,Städtische Beteiligung an Unternehmungen,Postulat,Abgeschrieben,21. November 2014,[ Senn Mario]
1,ZKB Jubiläums-Dividende,Interpellation,Beantwortet,6. November 2019,"[ Mäder Gabriel, Schanz Simon, , Stefan Neubert (Mitunterzeichner/in)\n Anke Würl (Mitunterzeichner/in)\n Martin Rüttimann (Mitunterzeichner/in)\n Walter Uebersax (Erstunterzeichner/in), , , ]"
2,Zwischennutzung Stadthausareal,Postulat,Überwiesen,7. September 2011,[ Clesle Hanspeter]
3,Anschluss des Quartiers Sonnenberg an den ÖV,Postulat,Abgelehnt,15. Dezember 2022,"[ Muthuthamby Kannathasan, Acar Sait]"
4,Berücksichtigung von lokalen Handwerker u. Unternehmen bei Beauftragungen durch die Stadt Adliswil,Interpellation,Beantwortet,16. Dezember 2022,[ Muthuthamby Kannathasan]
5,Sichere Schulwege im Soodstrassenbereich,Interpellation,Beantwortet,21. September 2011,[ Baldegger Harry]
6,Geplante Verkehrsentwicklung,Interpellation,Beantwortet,2. Oktober 2019,"[ Baldegger Harry, Mäder Gabriel, Melliger Heinz, , Stefan Neubert (Erstunterzeichner/in)\n Anke Würl (Mitunterzeichner/in)\n Walter Uebersax (Mitunterzeichner/in), , ]"
7,Reorganisation der Schule Adliswil,Interpellation,Beantwortet,17. November 2014,"[Fredi Morf (keine Funktion)\n Heidi Jucker (keine Funktion), ]"
8,Elternmitwirkung bei der Schülerzuteilung,Postulat,Erledigt,6. November 2014,[ Senn Mario]
9,Sauberkeit im öffentlichen Raum,Anfrage,Beantwortet,10. Februar 2016,[Dr. Liedtke Wolfgang]


In [8]:
def namenliste_reinigen(liste):
    
    if liste is None:
        return liste
    
    else:
        neueliste = []
        
        for element in liste:
            neueselement = element.strip()
            
            splitted_elemente = neueselement.split("\n")
        
            for splitted_element in splitted_elemente:
                cleaned_element = splitted_element.replace("(Mitunterzeichner/in)", "").replace('(Erstunterzeichner/in)',"").replace('(keine Funktion)',"").replace('(Verfasser/in)',"").replace('(Ersatzurheber/in)',"").replace('(Zweitunterzeichner/in)',"").replace('Dr. ',"").strip()

                if cleaned_element != "":
                    neueliste.append(cleaned_element)
    
    return neueliste

In [9]:
df['Namen-neu'] = df['Namen'].apply(namenliste_reinigen)

In [10]:
df['Namen-neu'].explode().value_counts()

Senn Mario                 30
Davide Loss                21
Liedtke Wolfgang           20
Corrodi Bernie             20
Frei Daniel                19
Oswald Marianne            17
Acar Sait                  17
Clesle Hanspeter           16
Walter Uebersax            14
Baldegger Harry            14
Mäder Gabriel              13
Muthuthamby Kannathasan    13
Fässler Thomas             12
Melliger Heinz             11
Anke Würl                  11
Etemi Xhelajdin            10
Heidi Jucker                9
Fredi Morf                  9
Daniela Morf                9
Bürgi Markus                8
Ueli Gräflein               8
Yannick Wettstein           7
Angelika  Sulser            7
Broggini Angela             7
Daniel Jud                  7
Schanz Simon                7
Schneider Daniel            6
Buchmann Reto               6
Buchmann-Bach Vera          6
Stefan Neubert              6
Helbling Silvia             6
Yilmaz Esen                 5
Iseli Thomas                5
Marty Fäss

In [11]:
setlocale(LC_TIME, 'de_DE.utf-8')
df['Datum'] = pd.to_datetime(df['Datum'], format='%d. %B %Y')
df.sort_values(by='Datum')

,Titel,Geschaeftsart,Status,Datum,Namen,Namen-neu
127,Parkierungskonzept im Tal,Postulat,Erledigt,2005-12-07,[Fredi Morf (keine Funktion)],[Fredi Morf]
128,Entwicklung im öffentlichen Verkehr,Interpellation,Beantwortet,2007-06-04,[Davide Loss (Verfasser/in)],[Davide Loss]
114,Integration des Quartiers Sonnenberg in das öffentliche Vekehrsnetz,Interpellation,Beantwortet,2007-06-27,[Davide Loss (Verfasser/in)],[Davide Loss]
113,Ersatzlösung für die Anbindung des Quartier Sonnenberg an das öffentliche Verkehrsnetz,Postulat,Erledigt,2007-12-03,[Davide Loss (keine Funktion)],[Davide Loss]
89,Lärmschutz Zürichstrasse,Interpellation,Beantwortet,2008-03-06,[Davide Loss (keine Funktion)],[Davide Loss]
88,Strassenbeleuchtung bei Bedarf,Interpellation,Beantwortet,2008-04-15,[ Fässler Thomas],[Fässler Thomas]
101,Neuer linksseitiger Sihluferweg mit Wegbeleuchtung,Motion,Erledigt,2008-08-17,[Fredi Morf (keine Funktion)],[Fredi Morf]
102,"Bauvorhaben der Schule, befristetes Aufstellen von 11 Schulcontainern",Interpellation,Erledigt,2008-09-03,[Fredi Morf (keine Funktion)],[Fredi Morf]
76,Energiebuchhaltung,Interpellation,Beantwortet,2008-11-26,[Yannick Wettstein (keine Funktion)],[Yannick Wettstein]
75,200-Watt-Gesellschaft,Postulat,Beantwortet,2009-02-05,[ Fässler Thomas],[Fässler Thomas]


In [12]:
df.to_csv('daten_adliswil.csv', index=False)